# Deep Neural Network

We attempt to implement a neural network for giving predictions. Appropriate articles explaining:
* [Regression with Neural Networks in PyTorch](https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379) (Medium)
* [How to create a neural network for regression with PyTorch](https://www.machinecurve.com/index.php/2021/07/20/how-to-create-a-neural-network-for-regression-with-pytorch/) (Machinecurve.com)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import mean_absolute_percentage_error as mape # usage : print(mape(pred, observed))

import datetime

import torch

path = 'C:/Users/sebir/Desktop/M2 Toulouse/Défi IA/'

# Import data

In [20]:
# Training set, X
# X = pd.read_csv(path + 'X_station_train.csv',
#                 parse_dates=['date'],
#                 infer_datetime_format=True)
# X.columns = ['number_sta', 'date', 'wind_speed', 'temperature', 'dew_point_temperature', 'humidity', 'wind_direction', 'precipitation', 'Id']

X_agg_filledNA = pd.read_csv(path + 'our_X_agg_filled_nearestNeighbours.csv',
                             parse_dates=['day'],
                             infer_datetime_format=True)

# Training set, Y
# Y = pd.read_csv(path + 'Y_train.csv',
#                 parse_dates=['date'],
#                 infer_datetime_format=True)

Y_filledwithNA = pd.read_csv(path + 'Y_train.csv',
                parse_dates=['date'],
                infer_datetime_format=True)

# Preprocess data

## `X` hourly to `X_agg`

We have to group the data by data in order to have something comparable with `Y`

In [3]:
X['day'] = X['date'].apply(lambda x: x.date())

In [4]:
X_agg = X.copy()
X_agg = X_agg.groupby(by = ['day', 'number_sta'], as_index = False).agg({
    "number_sta"            : "first",
    "wind_speed"            : "median",
    "temperature"           : "median",
    "dew_point_temperature" : "median",
    "humidity"              : "median",
    "wind_direction"        : "median",
    "precipitation"         : "sum"
})

We firstly drop all the NAs.

In [5]:
X_agg.dropna(inplace = True)
Y = Y.loc[X_agg.index]

Y.dropna(inplace = True)
X_agg = X_agg.loc[Y.index]

Adding the month before deleting the date, since we have this information in `X_test`:

In [6]:
X_agg['month'] = X_agg['day'].apply(lambda x: x.month)

We then delete the Id in `Y` which won't be helpful

In [7]:
X_agg.drop(['day'], axis = 1, inplace = True)
Y.drop(['date', 'Id'], axis = 1, inplace = True)

In [8]:
X_agg

,number_sta,wind_speed,temperature,dew_point_temperature,humidity,wind_direction,precipitation,month
0,14066001,3.695,281.325,278.625,90.15,142.0,0.2,1
2,14137001,5.830,281.155,278.905,93.50,140.0,0.0,1
3,14216001,5.010,280.630,278.815,91.25,155.5,0.6,1
7,14372001,5.650,280.600,278.465,89.45,158.5,0.2,1
11,14578001,5.230,280.195,278.760,95.90,160.0,0.8,1
...,...,...,...,...,...,...,...,...
183739,86027001,8.590,286.140,285.065,90.85,240.0,5.9,12
183742,86137003,3.765,286.450,284.880,88.90,227.5,3.4,12
183743,86165005,6.265,286.110,284.320,87.65,245.5,5.6,12
183745,91200002,4.380,286.070,284.555,93.45,233.5,5.6,12


## `X_agg_filledNA`
We took `X_agg` and we filled the gaps with data from nearests neighbours. Preprocessing data for coherence with `Y`:

Adding the month before deleting the date, since we have this information in `X_test`:

In [21]:
X_agg_filledNA['month'] = X_agg_filledNA['day'].apply(lambda x: x.month)
X_agg_filledNA.drop('day', axis = 1, inplace = True)

In [22]:
Y_filledwithNA.dropna(inplace = True)
X_agg_filledNA = X_agg_filledNA.loc[Y_filledwithNA.index]

<center><h1>Neural network

## Imports and create the classes

In [61]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

class MeteoDataset(torch.utils.data.Dataset):
    '''
    Prepare the dataset for regression
    '''

    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            # Apply scaling if necessary
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

    
class MLP(nn.Module):
    '''
    Multilayer Perceptron for regression.
    '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 100), # size of the input is here
            nn.LeakyReLU(),
            nn.Linear(100, 1)
        )


    def forward(self, x):
        '''
        Forward pass
        '''
        return self.layers(x)

torch.manual_seed(42);

## Load data set, set up optimizer

In [62]:
# HYPERPARAMETERS
batchSize = 100
learningRate = 1e-2
nbEpoch = 300

In [63]:
from sklearn.model_selection import train_test_split

X = X_agg_filledNA.to_numpy()
y = Y_filledwithNA['Ground_truth'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Training set
dataset = MeteoDataset(X_train, y_train)
trainloader = torch.utils.data.DataLoader(dataset,
                                          batch_size = batchSize,
                                          shuffle = True,
                                          # num_workers=1
                                         )
# Test set
testset = MeteoDataset(X_test, y_test)
testloader = torch.utils.data.DataLoader(testset)

# Reinitialize completely
try:
    del mlp
    del loss_function
    del optimizer
except:
    pass


# Initialize the MLP
mlp = MLP()

# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr = learningRate)

## Run

In [ ]:
# Run the training loop
trainLosses = []
testLosses = []
i = 1
for epoch in range(nbEpoch):
    print(f'Starting epoch {epoch+1}/{nbEpoch}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    # for i, data in enumerate(trainloader, start = 1):
    for data in trainloader:

        # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 100 == 0:
            # print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss))
            trainLosses.append([i, current_loss])
            current_loss = 0.0
        
        if i % 10000 == 0:
            # Test set to see if overfitting or not
            print("Proceeding to test...")
            outputs = []
            for data in testloader:

                    # Get and prepare inputs
                    inputs, _ = data
                    inputs = inputs.float()

                    # Perform forward pass
                    outputs.append(mlp(inputs).detach().item())
            # Add score to history
            testLosses.append([i, mape(outputs, y_test)])
            
        i += 1
        

# Process is complete.
print('Training process has finished.')

# Plotting errors
trainLosses = np.array(trainLosses)
testLosses = np.array(testLosses)

plt.plot(trainLosses[:, 0], trainLosses[:, 1]);
plt.title('Evolution of training error');
plt.show()
plt.plot(testLosses[:, 0], testLosses[:, 1]);
plt.title('Evolution of test error');
plt.show()

Starting epoch 1/300
Starting epoch 2/300
Starting epoch 3/300
Starting epoch 4/300
Starting epoch 5/300
Starting epoch 6/300
Starting epoch 7/300
Starting epoch 8/300
Starting epoch 9/300
Proceeding to test...
Starting epoch 10/300
Starting epoch 11/300
Starting epoch 12/300
Starting epoch 13/300
Starting epoch 14/300
Starting epoch 15/300
Starting epoch 16/300
Starting epoch 17/300
Starting epoch 18/300
Proceeding to test...
Starting epoch 19/300
Starting epoch 20/300
Starting epoch 21/300
Starting epoch 22/300
Starting epoch 23/300
Starting epoch 24/300
Starting epoch 25/300
Starting epoch 26/300
Starting epoch 27/300
Proceeding to test...
Starting epoch 28/300
Starting epoch 29/300
Starting epoch 30/300
Starting epoch 31/300
Starting epoch 32/300
Starting epoch 33/300
Starting epoch 34/300
Starting epoch 35/300
Starting epoch 36/300
Proceeding to test...
Starting epoch 37/300
Starting epoch 38/300
Starting epoch 39/300
Starting epoch 40/300
Starting epoch 41/300
Starting epoch 42/3

## Predictions

In [52]:
outputs = []
for i, data in enumerate(testloader, 0):

        # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        # Perform forward pass
        outputs.append(mlp(inputs).detach().item())
        
mape(outputs, y_test)

3.58359102989583